## RNN
- 시계열 데이터 및 자연어 처리하는데 주로 사용
- LSTM (Long short term memory), GRU(Gated recurrent unit)등이 언어 모델링(language modeling), 텍스트 감정 분석(text sentiment analysis, 기계 번역(machine translation) 등의 분야에서 활발하게 이용되고 있음
- GRU(Gated recurrent unit)은 시계열 데이터 속 벡터 사이의 정보 전달량을 조절함으로서 기울기를 적정하게 유지하고 문장 앞부분의 정보가 끝까지 도달할 수 있도록 도와줌
    - 업데이트 게이트(이전 은닉 벡터가 지닌 정보를 새로운 은닉 벡터가 얼마나 유지할지)와 리셋 게이트라는 개념이 존재
    
### RNN 종류
- 1) one to one (일대일) : 일반적인 신경망, CNN과 동일
- 2) one to many (일대다) : 이미지를 보고 이미지 안의 상황을 글로 설명하는 문제
- 3) many to one (다대일) : 감정 분석 같이 순차적인 데이터를 보고 하나를 내는 경우
- 4) many to many1 (다대다1) : 챗봇과 기계 번역 같이 순차적인 데이터를 보고 순차적인 데이터를 출력하는 문제
- 5) many to many2 (다대다2) : 비디오 분류 같이 매 프레임을 레이블링 할 때 사용됨
![rnn](readme_image/rnn.png)

### 영화 리뷰 감정 분석
- 다대일
- IMDB 데이터
    - 긍정은 2, 부정은 1
    - 영화 리뷰 데이터 5만 건

### 데이터 전처리
- 텍스트를 전처리 과정을 거쳐 숫자로 나타내어야 함.

- (1) 토크나이징 (Tokenization)
    - 언어의 최소 단위인 토큰으로 나누는 것
    - split() 함수 또는 Spacy 오픈 소스 라이브러리 사용 가능
    - 데이터셋의 모든 단어 수만큼의 벡터를 담는 사전을 정의
        - ex) quick brown fox jumps over the lazy dog => ['quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

- (2) 워드임베딩(word embedding)
    - 문장 속 모든 토큰을 각각의 벡터로 나타내주는 것



In [1]:
## 1. 라이브러리 import
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets # 자연어 처리 데이터셋

# 하이퍼 파라미터 설정
batch_size =64
lr = 0.001
epochs = 10

# CUDA 여부 및 DEVICE 체크
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")


In [3]:
## 2. 데이터셋 확보

# sequential 파라미터를 이용해 데이터셋이 순차적인 데이터셋인지 명시해줌
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first = True)

# IMDB Train set, Test set 만들기
trainset, testset = datasets.IMDB.splits(TEXT, LABEL) 

# 워드 임베딩에 필요한 단어 사전 만들기 (word vocabulary)
# min freq 미만 인 단어들은 Unknown이라는 뜻의 unk라는 토큰으로 대체됨.
TEXT.build_vocab(trainset, min_freq = 5) #  min freq는 최소 몇번 등장한 단어마만을 사용하겠다는 뜻
LABEL.build_vocab(trainset)

# train, validation, test data 배치별로 뽑아주는 getnerator
trianset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset), batch_size = batch_size, shuffle =True, repeat = False
)

# 사전 속 단어들의 갯수와 레이블 수 정해줌
vocab_size = len(TEXT.vocab)
n_classes = 2

print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d" %(len(trainset), len(valset), len(testset), vocab_size, n_classes))


[학습셋]: 25000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [10]:
# 3. RNN 모델 구현
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p = 0.2):
        super(BasicGRU, self).__init__()
        print('Building Basic GRU Model...')
        self.n_layers = n_layers
        ## embed 결과 -> quick:[0.1, 0.2], lazy[0.2, 0.8], dog[0.8, 0.9] 
        self.embed = nn.Embedding(n_vocab, embed_dim) # 사전에 등재된 단어수, 임베딩된 단어 텐서가 지니는 차원 값
        
        # hyper parameter 값들
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        # gru 모델 넣기. embed차원, hidden layer 차원, layer들
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers, batch_first=True)
        
        # 모델 최종 Dense 신경망
        self.out = nn.Linear(self.hidden_dim, n_classes) # 입력된 텐서를 nn 신경망을 통과시켜 예측
        
    def forward(self, x):
        x = self.embed(x) # embed 결과
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번쨰 은닉벡터
        x, _ = self.gru(x, h_0) # 첫번째 은닉벡터인 h_0을 gru에 넣으면 은닉 벡터들이 시계열 배열 형태로 반환
        
        # (batch_size, 입력x의 길이, hidden_dim) 텐서들 => 영화데이터들이 압축된 은닉 벡터
        h_t = x[:,-1,:]  
        self.dropout(h_t)
        logit = self.out(h_t) # self.out 신경망에 입력해 결과를 출력
        return logit

    # 신경망 모듈의 가중치 정보들을 반복자 형태로 반환
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data # nn.GRU 모듈의 첫번쨰 가중치 텐서를 추출
        
        # 그 후 NEW 함수를 호출해 모델의 가중치와 같은 모양인 텐서로 변환한후 zero()로 초기화
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_() # zero_() : 모든값을 0으로 초기화


In [11]:
# 학습함수
def train(model, optimizer, train_iter):
    model.train()
    # 훈련 데이터 b=number , batch=영화평 데이터와, label들(긍부정)
    for b, batch in enumerate(train_iter): 
        
        # x는 영화평 데이터, y는 label들
        # batch.text와 batch.label로 나뉨
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블값을 0과 1로 변환
        
        # 함수로 기울기를 0으로 초기화
        optimizer.zero_grad() 
        logit = model(x) # 학습 데이터x를 모델에 입력해 예측값(logit)을 구함
        
        # 오차 및 학습 진행
        loss = F.cross_entropy(logit, y)  
        loss.backward()
        optimizer.step()
        
# 평가함수
def evaluate(model, val_iter):
    model.eval() # evaluate mode 변경
    corrects, total_Loss =0, 0 # 정답 수와, 총 loss

    # test data
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블값을 0과 1로 변환
        logit = model(x)
        
        # loss 구하기, batch별 loss가 아닌 전체 데이터에 대한 loss 합 이기에 sum
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss +=loss.item()
        
        # 정답수
        corrects += (logit.max(1)[1].view(y.size()).data ==y.data).sum()
    
    # 총 loss, accuracy return
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0*corrects/size
    return avg_loss, avg_accuracy


In [ ]:
## 모델 생성 및 학습 진행
# 모델 생성 (은닉벡터의 차원값 : 256,  임베딩된 토큰의 차원값 : 128)
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_val_loss = None
for e in range(1, epochs+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" %(e, val_loss, val_accuracy))
    
    # 학습 오차가 가장 작은 것이 아닌 검증 오차가 젤 작은 것.
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        # 디렉토리 없으면 만들기
        if not os.path.isdir("model"):
            os.makedirs("model")
        
        # 모델 저장 및 loss 저장
        torch.save(model.state_dict(), 'model/sentiment_classification.pt')
        best_val_loss = val_loss

# 저장된 모델 load 및 최종 테스트
model.load_state_dict(torch.load('model/sentiment_classification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도 : %5.2f' %(test_loss, test_acc))

Building Basic GRU Model...
